## Ch09 대규모 데이터의 처리를 위한 Sparse matrix 사용

이 책에서 약 1600개의 영화에 대한 100000개의 평점을 포함하고 있는 데이터이다.  
현실에서 다루어야 하는 데이터는 이보다 훨씬 더 크다.  
Movielens 20M 데이터는 약 2000만 개의 데이터를 포함하고 있는데 이런 데이터를 처리할 때 흔하게 발생하는 문제는 메모리의 한계이다.  
앞에서는 데이터를 full matrix로 변환하여 처리하였는데 이 데이터를 같은 방식을 적용하면 너무 많은 원소의 행렬이 생기게 되어 메모리의 한계로 처리할 수 없다.  
이 때 사용하는 것이 sparse matrix 로 이는 추천 시스템에서 사용되는 대부분의 데이터에서 full matrix로 변환했을 때 많은 원소가 비어있는 희박행렬을 말한다.

### 9.1 Sparse matrix의 개념과 Python에서의 사용

다룰 데이터가 sparse matrix 라면 전체 행렬을 저장하는 대신 행렬의 원소 중에서 실제로 값을 갖는 것만 인덱스와 같이 저장하는 것이 더 효율적이다.  
이 방식은 데이터가 희박할수록 저장공간이 절약되는 정도가 커진다.

sparse matrix의 단점은 데이터를 저장하거나 읽어올 때마다 값이 존재하는지 확인해서 그에 맞는 처리를 해야 하기 때문에 데이터 처리의 오버헤드 비용이 크다.  
따라서 데이터가 희박하지 않은 경우 sparse matrix를 사용하면 속도저하의 단점이 크게 부각되므로 데이터가 희박한 경우에만 사용하는 것이 낫다.

다음은 sparse matrix 간단한 사용 예이다.

In [ ]:
import numpy as numpy
import pandas as pd
from scipy.sparse import csr_matrix

